In [ ]:
pip install neo4j

In [ ]:
from neo4j import GraphDatabase

class Neo4jScienceQueries:
    def __init__(self, uri, username, password):
        self.driver = GraphDatabase.driver(uri, auth=(username, password))

    def close(self):
        self.driver.close()

    def run_query(self, query):
        with self.driver.session() as session:
            result = session.run(query)
            return result.data()

    def create_science_community(self):
        query = """
        MATCH (k:Keyword)
        WHERE k.Name IN [
          'Physics', 
          'Chemistry', 
          'Biology', 
          'Computer Science', 
          'Political Science',
          'Agricultural and Food Sciences'
        ]
        WITH collect(k) AS Keywords
        MERGE (c:Community {name: "Science Community"})
        WITH c, Keywords
        UNWIND Keywords AS keyword
        MERGE (c)-[:INCLUDES]->(keyword)
        RETURN c.name AS Community, count(Keywords) AS KeywordCount
        """
        return self.run_query(query)

    def find_related_venues(self):
        query = """
        MATCH (c:Community {name: "Science Community"})-[:INCLUDES]->(k:Keyword)<-[:About]-(p:Paper)
        WITH c, collect(DISTINCT p) AS communityPapers
        
        // For conferences
        MATCH (conf:Conference)<-[:Of]-(edition:Conference_Edition)<-[:Published_in]-(paper:Paper)
        WITH c, communityPapers, conf, collect(DISTINCT paper) AS confPapers
        WHERE size(confPapers) > 0 AND 
              size([p IN confPapers WHERE p IN communityPapers]) >= 0.9 * size(confPapers)
        MERGE (conf)-[:RELATED_TO]->(c)
        
        // For journals (separate handling)
        WITH c, communityPapers
        MATCH (journal:Journal)<-[:Published_in]-(paper:Paper)
        WITH c, communityPapers, journal, collect(DISTINCT paper) AS journalPapers
        WHERE size(journalPapers) > 0 AND 
              size([p IN journalPapers WHERE p IN communityPapers]) >= 0.9 * size(journalPapers)
        MERGE (journal)-[:RELATED_TO]->(c)
        
        RETURN "Related venues identified" AS Result
        """
        return self.run_query(query)

    def find_top_papers(self):
        query = """
        MATCH (c:Community {name: "Science Community"})
        MATCH (venue)-[:RELATED_TO]->(c)
        WHERE venue:Conference OR venue:Journal
        WITH c, collect(venue) AS relatedVenues
        
        // For conferences, we need to go through Conference_Edition
        MATCH (conf:Conference)
        WHERE conf IN relatedVenues
        MATCH (paper:Paper)-[:Published_in]->(edition:Conference_Edition)-[:Of]->(conf)
        WITH c, relatedVenues, collect(DISTINCT paper) AS confPapers
        
        // For journals
        MATCH (journal:Journal)
        WHERE journal IN relatedVenues
        MATCH (paper:Paper)-[:Published_in]->(journal)
        WITH c, confPapers, collect(DISTINCT paper) AS journalPapers
        
        // Combine all community papers
        WITH c, confPapers + journalPapers AS communityPapers
        
        // Count citations from papers within the community
        UNWIND communityPapers AS paper
        MATCH (paper)-[:Cited_by]->(citingPaper)
        WHERE citingPaper IN communityPapers
        WITH paper, count(DISTINCT citingPaper) AS citationCount
        ORDER BY citationCount DESC
        LIMIT 100
        
        // Create relationship between these top papers and the community
        MATCH (c:Community {name: "Science Community"})
        MERGE (paper)-[:TOP_PAPER_IN {citations: citationCount}]->(c)
        
        RETURN "Top papers identified" AS Result
        """
        return self.run_query(query)

    def identify_reviewers_and_gurus(self):
        query = """
        MATCH (c:Community {name: "Science Community"})
        MATCH (paper:Paper)-[:TOP_PAPER_IN]->(c)
        MATCH (author:Author)-[:Writes]->(paper)
        
        // Mark all authors of top papers as potential reviewers
        MERGE (author)-[:POTENTIAL_REVIEWER_FOR]->(c)
        
        // Count papers per author to identify gurus (authors with at least 2 top papers)
        WITH c, author, count(DISTINCT paper) AS paperCount
        WHERE paperCount >= 2
        
        // Mark these authors as gurus
        MERGE (author)-[:GURU_OF {top_paper_count: paperCount}]->(c)
        
        RETURN count(DISTINCT author) AS GuruCount
        """
        return self.run_query(query)

    def run_all_queries(self):
        print("Creating Science Community...")
        result1 = self.create_science_community()
        print(result1)
        
        print("\nFinding related venues...")
        result2 = self.find_related_venues()
        print(result2)
        
        print("\nIdentifying top papers...")
        result3 = self.find_top_papers()
        print(result3)
        
        print("\nIdentifying reviewers and gurus...")
        result4 = self.identify_reviewers_and_gurus()
        print(result4)

In [ ]:
if __name__ == "__main__":
    # Replace with your Neo4j connection details
    uri = "neo4j://localhost:7687"
    username = "neo4j"
    password = "pass1234"
    
    app = Neo4jScienceQueries(uri, username, password)
    
    try:
        app.run_all_queries()
    finally:
        app.close()